# Generate stores & customers for the demo

## Setup the environment

### imports

In [2]:
import logging
import os
import pickle
import random
import itertools
import boto3
import botocore

# Demo
import pandas as pd
import pymorton as pm
from faker import Faker
from faker.providers import BaseProvider

# DB
import v3io as v3c
import v3io_frames as v3f

### Define environment variables

In [6]:
# Stores
NUMBER_OF_STORES=50
STORES_TABLE='stores'
ACCURACY=20
BASE_ACCURACY=14
DATA_DIR = os.getenv('DATA_DIR','/v3io/users/' + os.getenv('V3IO_USER','iguazio') + '/lbr_data') 

# Customers
NUMBER_OF_CUSTOMERS=5000
CUSTOMERS_TABLE='customers'
os.environ['CUSTOMERS_DATA_FILE_DIR'] = DATA_DIR
os.environ['CUSTOMERS_DATA_FILE_PATH'] = os.path.join(os.environ['CUSTOMERS_DATA_FILE_DIR'], 'data_train_n.csv')
!mkdir -p ${CUSTOMERS_DATA_FILE_DIR}

In [7]:
!curl -L "iguazio-sample-data.s3.amazonaws.com/data_train_n.csv" > ${CUSTOMERS_DATA_FILE_PATH}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  399M  100  399M    0     0  16.4M      0  0:00:24  0:00:24 --:--:-- 17.7M


### Open DB client

In [8]:
client = v3f.Client('framesd:8081')

## Generate Stores

### Location provider

In [9]:
class LocationProvider(BaseProvider):
    '''
    Creates locations within base_location

    Uses QuadTree for Geohashing
        @{http://tech.taskrabbit.com/blog/2015/06/09/elasticsearch-geohash-vs-geotree/}
        @{http://mapzen.github.io/leaflet-spatial-prefix-tree/}
        @{http://blog.notdot.net/2009/11/Damn-Cool-Algorithms-Spatial-indexing-with-Quadtrees-and-Hilbert-Curves}
    '''    
    def location(self, location_base: str, base_acc=10, acc=20):
        coordinates = location_base[:base_acc]
        for i in range(acc-len(coordinates)):
            coordinates += str(random.randint(0, 3))
        return coordinates

Add the location provider as a faker provider

In [10]:
faker = Faker()
faker.add_provider(LocationProvider)

### Define scenario environment (locations)

In [12]:
london_city = (51.514926, -0.089580)
london_city_south = (51.501593, -0.094942)
london_west = (51.512309, -0.128966)
london_south_west = (51.495022, -0.162268)

london_coordinates = []
london_coordinates.append(london_city)
london_coordinates.append(london_city_south)
london_coordinates.append(london_west)
london_coordinates.append(london_south_west)

london_coordinates_qt = list(map(lambda cooridnate: pm.interleave_latlng(*cooridnate), london_coordinates))
#london_coordinates_qt

### Define Store class

In [13]:
class Store(object):
    def __init__(self, store_id: int, name: str, location: str):
        self.store_id = store_id if store_id is not None else random.randint(1, int(os.genev('NUMBER_OF_STORES', 500)))
        self.location = location
        (self.lat, self.long) = pm.deinterleave_latlng(location)
        self.store_name = name
        self.customers = 0
    
    def json(self):
        json_store = {
            'id': self.store_id,
            'name': self.store_name,
            'location': self.location,
            'latitude': self.lat,
            'longitude': self.long,
            'customers': self.customers
        }
        return json_store

### Create stores

In [14]:
def create_stores(faker, number_of_stores: int=500):
    remaining = number_of_stores
    stores = []
    while remaining > 0:
        # Create stores
        stores_batch = [Store(faker.msisdn(), faker.company(), faker.location(random.choice(london_coordinates_qt), BASE_ACCURACY, ACCURACY)).json() 
              for i in range(remaining)]
        
        # Count for duplicate locations
        locations = list(map(lambda store: store['location'], stores_batch))
        locations = list(dict.fromkeys(locations))
        remaining -= len(locations)
        
        # Update stores list
        stores.append(stores_batch)

    stores = list(itertools.chain.from_iterable(stores))
    stores = pd.DataFrame.from_records(stores)
    stores = stores.set_index(['location'])
    stores = stores[~stores.index.duplicated(keep='first')]
    return stores

In [26]:
try:
    client.delete('kv', STORES_TABLE, if_missing=1)
except:
    print("Table ", STORES_TABLE, " does not exist")

Table  stores  does not exist


In [28]:
stores = create_stores(faker, NUMBER_OF_STORES)

### Upload to DB 

In [29]:
# client.delete('kv', STORES_TABLE)
client.write('kv', STORES_TABLE, stores)

## Generate customers

In [30]:
class Customer(object):
    def __init__(self, id=0, location='0', products=""):
        self.id = id
        self.location = location
        self.products = products
        
    def json(self):
        json_customer = {
            'id': self.id,
            'location': self.location,
            'products': self.products
        }
        return json_customer

In [31]:
def create_customers(customers_data: pd.DataFrame, number_of_customers: int):
    available_customer = list(customers_data.user_id)
    customers = [Customer(id=i,
                          products=customers_data.loc[customers_data['user_id'] == random.choice(available_customer)]\
                                    .drop(['user_id', 'Unnamed: 0'], axis=1)\
                                    .to_json()).json() for i in range(number_of_customers)]
    customers = pd.DataFrame.from_records(customers)
    customers.set_index(['id'])
    return customers

In [32]:
# Load customers seed data
customers_data = pd.read_csv(os.environ['CUSTOMERS_DATA_FILE_PATH'])

# Create actual customers
customers = create_customers(customers_data, NUMBER_OF_CUSTOMERS)

,id,location,products
0,0,0,"{""user_product_reordered_ratio"":{""49732"":1.0,""..."
1,1,0,"{""user_product_reordered_ratio"":{""50882"":1.0,""..."
2,2,0,"{""user_product_reordered_ratio"":{""148007"":1.0,..."
3,3,0,"{""user_product_reordered_ratio"":{""17403"":1.0,""..."
4,4,0,"{""user_product_reordered_ratio"":{""31534"":1.0,""..."


## Upload to DB

In [36]:
try:
    client.delete('kv', CUSTOMERS_TABLE, if_missing=1)
except:
        print("Table ", CUSTOMERS_TABLE, " does not exist")

In [37]:
try:
    client.write('kv', CUSTOMERS_TABLE, dfs=customers)
except:
    print("Failed to insert some rows into ", CUSTOMERS_TABLE)

Failed to insert some rows into  customers


In [41]:
print(f'Customers table sized: {client.read("kv", CUSTOMERS_TABLE).shape}')

Customers table sized: (4979, 3)


## Generate predictions

In [39]:
try:
    client.delete('tsdb', 'predictions', if_missing=1)
except:
    print("Table predictions does not exist")


Table predictions does not exist


In [40]:
os.system("tsdbctl create -c bigdata -t predictions -r 1/s")

0